<h1><center>ABBA: Lyrics Emotions</center></h1>
<img src="../images/gettyimages-abba.jpeg" width="360" align="center"/>

This notebook details all the steps performed to create a **song lyrics dataset**. The process is limited to studio albums and includes only English songs - instrumental, remixed, live songs are left out.

- Some documentation: https://en.wikipedia.org/wiki/ABBA_discography#Studio_albums

In [138]:
import pandas as pd
import numpy as np
import datetime
import lyricsgenius

Get a token to download song lyrics
- lyricsgenius: https://pypi.org/project/lyricsgenius/
- api client: https://genius.com/api-clients
- https://towardsdatascience.com/song-lyrics-genius-api-dcc2819c29

In [5]:
token = 'cxk4nYrZcSzjbZ_xsGVItrJgsZbU5tkZlMn6SQLHDOjMd7SfkWa4CS4Zlta2h_uL'
genius = lyricsgenius.Genius(token)

# genius.verbose = False # Turn off status messages
genius.remove_section_headers = True # Remove section headers (e.g. [Chorus]) from lyrics when searching
genius.skip_non_songs = False # Include hits thought to be non-songs (e.g. track lists)
genius.excluded_terms = ["(Remix)", "(Live)"] # Exclude songs with these words in their title

In [28]:
# Function used to cast dates (downloaded from Genius as dict) in datetime format
def get_release_date(date_dict):
    year, month, day = [str(item) for key, item in date_dict.items()]
    year, month, day = year.rjust(4, '0'), month.rjust(2, '0'), day.rjust(2, '0')
    return datetime.datetime.strptime(year+month+day, '%Y%m%d')

# Function to create a lyrics dataset based on a given album name
def lyrics_df(album_name, id_album):
    album = genius.search_album(name=album_name, artist="ABBA", get_full_info=True, text_format=True)
    album_dict = album.to_dict()
    n_tracks = len(album_dict['tracks'])
    lyrics = (
        [(album_dict['tracks'][i]['song']['title'], album_dict['tracks'][i]['song']['lyrics']) for i in range(n_tracks)]
    )
    df_out = pd.DataFrame(
        data={
            'id': id_album,
            'album': album.full_title,
            'release_date': [get_release_date(album_dict['release_date_components'])]*n_tracks,
            'n_tracks_original': n_tracks,
            'id_track': range(1, 1+n_tracks),
            'track': [single for single, _ in lyrics],
            'lyrics': [text for _, text in lyrics]
        }
    )
    print(album_dict['url'])
    return df_out

The download is done album by album (8 in total) in order to analyse each single collection.

## 1. Ring Ring
Released: 26 March 1973
- https://genius.com/albums/Abba/Ring-ring-international-edition
- https://en.wikipedia.org/wiki/Ring_Ring_(album)

In [7]:
# Download the album
album = genius.search_album(
    name='Ring Ring (International Edition)', artist="ABBA", get_full_info=True, text_format=True
)

Searching for "Ring Ring (International Edition)" by ABBA...


In [17]:
# Explore its features
album_dict = album.to_dict()
album_dict

{'_type': 'album',
 'api_path': '/albums/142507',
 'cover_art_thumbnail_url': 'https://images.genius.com/582b96992dd6faf042351a975f09dc2c.300x300x1.png',
 'cover_art_url': 'https://images.genius.com/582b96992dd6faf042351a975f09dc2c.999x999x1.png',
 'full_title': 'Ring Ring (International Edition) by ABBA',
 'id': 142507,
 'name': 'Ring Ring (International Edition)',
 'name_with_artist': 'Ring Ring (International Edition) (artist: ABBA)',
 'release_date_components': {'year': 1973, 'month': 3, 'day': 26},
 'url': 'https://genius.com/albums/Abba/Ring-ring-international-edition',
 'artist': {'_type': 'artist',
  'api_path': '/artists/33385',
  'header_image_url': 'https://images.genius.com/005a5d703ff7cff6bf1e56749f0d5ae7.1000x333x1.png',
  'id': 33385,
  'image_url': 'https://images.genius.com/30fd2acb597d61320415bf843a59564d.1000x1000x1.jpg',
  'index_character': 'a',
  'is_meme_verified': False,
  'is_verified': False,
  'name': 'ABBA',
  'slug': 'Abba',
  'url': 'https://genius.com/art

In [12]:
# Get the Genius URL of the album
print(album_dict['url'])

https://genius.com/albums/Abba/Ring-ring-international-edition


In [18]:
# Check the number of tracks
n_tracks = len(album_dict['tracks'])
n_tracks

15

In [19]:
# Get the song lyrics
lyrics = (
    [(album_dict['tracks'][i]['song']['title'], album_dict['tracks'][i]['song']['lyrics']) for i in range(n_tracks)]
)
lyrics
# lyrics is a list of the songs in the album, each element is a tuple defined as ({song name}, {song lyrics})

[('Ring Ring',
  "I was sitting by the phone\nI was waiting all alone\nBaby by myself I sit and wait and wonder about you\nIt's a dark and dreary night\nSeems like nothing's going right\nWon't you tell me honey how can I go on here without you?\n\nYes I'm down and feeling blue\nAnd I don't know what to do, oh-oh\n\nRing, ring, why don't you give me a call?\nRing, ring, the happiest sound of them all\nRing, ring, I stare at the phone on the wall\nAnd I sit all alone impatiently\nWon't you please understand the need in me\nSo, ring, ring, why don't you give me a call?\nSo, ring, ring, why don't you give me a call?\n\nYou were here and now you're gone\nHey, did I do something wrong?\nI just can't believe that I could be so badly mistaken\nWas it me or was it you?\nTell me, are we really through?\nWon't you hear me cry and you will know that my heart is breaking\nPlease forgive and then forget\nOr maybe darling better yet, oh-oh\n\nRing, ring, why don't you give me a call?\nRing, ring, the

In [83]:
[track for track, _ in lyrics]

['Ring Ring',
 'Another Town, Another Train',
 'Disillusion',
 'People Need Love',
 'I Saw It in the Mirror',
 'Nina, Pretty Ballerina',
 'Love Isn’t Easy (But It Sure Is Hard Enough)',
 'Me and Bobby and Bobby’s Brother',
 'He Is Your Brother',
 'She’s My Kind of Girl',
 'I Am Just a Girl',
 'Rock ’n Roll Band',
 'Merry-Go-Round',
 'Santa Rosa',
 'Ring Ring (Bara du Slog en Signal) [Swedish Version]']

In [29]:
# Collect the relevant information in a df
df_ring_ring = pd.DataFrame(
    data={
        'id': 1,
        'album': album.full_title,
        'release_date': [get_release_date(album_dict['release_date_components'])]*n_tracks,
        'n_tracks_original': n_tracks,
        'id_track': range(1, 1+n_tracks),
        'track': [single for single, _ in lyrics],
        'lyrics': [text for _, text in lyrics]
    }
)

In [30]:
# Drop the Swedish song
df_ring_ring = df_ring_ring.iloc[:-1]
df_ring_ring['n_tracks'] = 14

In [31]:
# Add some additional notes
df_ring_ring['notes'] = np.nan

df_ring_ring.loc[df_ring_ring.id_track==1, 'notes'] = 'Third at the 1973 Melodifestivalen'
df_ring_ring.loc[df_ring_ring.id_track==4, 'notes'] = 'First single ever'
df_ring_ring.loc[df_ring_ring.id_track==13, 'notes'] = '2001 CD edition bonus tracks'
df_ring_ring.loc[df_ring_ring.id_track==14, 'notes'] = '2001 CD edition bonus tracks'

In [32]:
df_ring_ring

,id,album,release_date,n_tracks_original,id_track,track,lyrics,n_tracks,notes
0,1,Ring Ring (International Edition) by ABBA,1973-03-26,15,1,Ring Ring,I was sitting by the phone\nI was waiting all ...,14,Third at the 1973 Melodifestivalen
1,1,Ring Ring (International Edition) by ABBA,1973-03-26,15,2,"Another Town, Another Train","Day is dawning and I must go\nYou're asleep, b...",14,NaN
2,1,Ring Ring (International Edition) by ABBA,1973-03-26,15,3,Disillusion,"Changing, moving in a circle\nI can see your f...",14,NaN
3,1,Ring Ring (International Edition) by ABBA,1973-03-26,15,4,People Need Love,"People need hope, people need loving\nPeople n...",14,First single ever
4,1,Ring Ring (International Edition) by ABBA,1973-03-26,15,5,I Saw It in the Mirror,"I saw it in the mirror, I saw it in my face\nT...",14,NaN
5,1,Ring Ring (International Edition) by ABBA,1973-03-26,15,6,"Nina, Pretty Ballerina",Every day in the morning on her way to the off...,14,NaN
6,1,Ring Ring (International Edition) by ABBA,1973-03-26,15,7,Love Isn’t Easy (But It Sure Is Hard Enough),Do you remember the first time and all of your...,14,NaN
7,1,Ring Ring (International Edition) by ABBA,1973-03-26,15,8,Me and Bobby and Bobby’s Brother,"Well, there was me and Bobby and Bobby's broth...",14,NaN
8,1,Ring Ring (International Edition) by ABBA,1973-03-26,15,9,He Is Your Brother,I was a fighter always looking for trouble\nAn...,14,NaN
9,1,Ring Ring (International Edition) by ABBA,1973-03-26,15,10,She’s My Kind of Girl,"Look at her face, it's a wonderful face\nAnd i...",14,NaN


## 2. Waterloo
Released: 4 March 1974
- https://genius.com/albums/Abba/Waterloo
- https://en.wikipedia.org/wiki/Waterloo_(album)

In [66]:
# Repeat the same initial steps for any album, using the function
df_waterloo = lyrics_df('Waterloo', 2)
df_waterloo

,id,album,release_date,n_tracks_original,id_track,track,lyrics
0,2,Waterloo by ABBA,1974-03-04,12,1,Waterloo,"My, my - at Waterloo, Napoleon did surrender\n..."
1,2,Waterloo by ABBA,1974-03-04,12,2,Sitting in the Palmtree,See me sitting in the palm tree\nLooking down ...
2,2,Waterloo by ABBA,1974-03-04,12,3,King Kong Song,Well I was looking at a movie on the TV last n...
3,2,Waterloo by ABBA,1974-03-04,12,4,Hasta Mañana,Where is the spring and the summer\nThat once ...
4,2,Waterloo by ABBA,1974-03-04,12,5,My Mama Said,Tried to sneak out without saying\nWith my lou...
5,2,Waterloo by ABBA,1974-03-04,12,6,Dance (While the Music Still Goes On),"Oh, my love it makes me sad\nWhy did things tu..."
6,2,Waterloo by ABBA,1974-03-04,12,7,"Honey, Honey","Honey, honey, how you thrill me, aha, honey ho..."
7,2,Waterloo by ABBA,1974-03-04,12,8,Watch Out,You move like a flame of fire\nYour eyes like ...
8,2,Waterloo by ABBA,1974-03-04,12,9,What About Livingstone?,Went to buy me a paper at the local newsstand\...
9,2,Waterloo by ABBA,1974-03-04,12,10,Gonna Sing You My Lovesong,You say she's been mad at you\nThen you say yo...


In [67]:
# Drop remixed songs
df_waterloo = df_waterloo.copy().iloc[:-1]
df_waterloo['n_tracks'] = 11

In [68]:
# Notes
df_waterloo['notes'] = np.nan
df_waterloo.loc[df_waterloo.id_track==1, 'notes'] = '1st at the Eurovision Song Contest on April 6, 1974'

In [69]:
df_waterloo

,id,album,release_date,n_tracks_original,id_track,track,lyrics,n_tracks,notes
0,2,Waterloo by ABBA,1974-03-04,12,1,Waterloo,"My, my - at Waterloo, Napoleon did surrender\n...",11,"1st at the Eurovision Song Contest on April 6,..."
1,2,Waterloo by ABBA,1974-03-04,12,2,Sitting in the Palmtree,See me sitting in the palm tree\nLooking down ...,11,NaN
2,2,Waterloo by ABBA,1974-03-04,12,3,King Kong Song,Well I was looking at a movie on the TV last n...,11,NaN
3,2,Waterloo by ABBA,1974-03-04,12,4,Hasta Mañana,Where is the spring and the summer\nThat once ...,11,NaN
4,2,Waterloo by ABBA,1974-03-04,12,5,My Mama Said,Tried to sneak out without saying\nWith my lou...,11,NaN
5,2,Waterloo by ABBA,1974-03-04,12,6,Dance (While the Music Still Goes On),"Oh, my love it makes me sad\nWhy did things tu...",11,NaN
6,2,Waterloo by ABBA,1974-03-04,12,7,"Honey, Honey","Honey, honey, how you thrill me, aha, honey ho...",11,NaN
7,2,Waterloo by ABBA,1974-03-04,12,8,Watch Out,You move like a flame of fire\nYour eyes like ...,11,NaN
8,2,Waterloo by ABBA,1974-03-04,12,9,What About Livingstone?,Went to buy me a paper at the local newsstand\...,11,NaN
9,2,Waterloo by ABBA,1974-03-04,12,10,Gonna Sing You My Lovesong,You say she's been mad at you\nThen you say yo...,11,NaN


## 3. ABBA
Released: 21 April 1975
- https://genius.com/albums/Abba/Abba
- https://en.wikipedia.org/wiki/ABBA_(album)

In [106]:
df_abba = lyrics_df('Abba', 3)
df_abba

Searching for "Abba" by ABBA...
https://genius.com/albums/Abba/Abba


,id,album,release_date,n_tracks_original,id_track,track,lyrics
0,3,ABBA by ABBA,1975-04-21,11,1,Mamma Mia,I've been cheated by you since I don't know wh...
1,3,ABBA by ABBA,1975-04-21,11,2,"Hey, Hey Helen","So at last, you're free\nIt's the way you want..."
2,3,ABBA by ABBA,1975-04-21,11,3,Tropical Loveland,"Come to my loveland, wander along\nBeautiful g..."
3,3,ABBA by ABBA,1975-04-21,11,4,S.O.S.,Where are those happy days? They seem so hard ...
4,3,ABBA by ABBA,1975-04-21,11,5,Man in the Middle,Did you see that man in the limousine?\nWith t...
5,3,ABBA by ABBA,1975-04-21,11,6,Bang-a-Boomerang,Making somebody happy is a question of give an...
6,3,ABBA by ABBA,1975-04-21,11,7,"I Do, I Do, I Do, I Do, I Do","Love me or leave me, make your choice but beli..."
7,3,ABBA by ABBA,1975-04-21,11,8,Rock Me,"Rock me, give me that kick now\nRock me, show ..."
8,3,ABBA by ABBA,1975-04-21,11,9,Intermezzo No. 1,
9,3,ABBA by ABBA,1975-04-21,11,10,I’ve Been Waiting for You,"I, I've been in love before\nI thought I would..."


In [107]:
df_abba

,id,album,release_date,n_tracks_original,id_track,track,lyrics
0,3,ABBA by ABBA,1975-04-21,11,1,Mamma Mia,I've been cheated by you since I don't know wh...
1,3,ABBA by ABBA,1975-04-21,11,2,"Hey, Hey Helen","So at last, you're free\nIt's the way you want..."
2,3,ABBA by ABBA,1975-04-21,11,3,Tropical Loveland,"Come to my loveland, wander along\nBeautiful g..."
3,3,ABBA by ABBA,1975-04-21,11,4,S.O.S.,Where are those happy days? They seem so hard ...
4,3,ABBA by ABBA,1975-04-21,11,5,Man in the Middle,Did you see that man in the limousine?\nWith t...
5,3,ABBA by ABBA,1975-04-21,11,6,Bang-a-Boomerang,Making somebody happy is a question of give an...
6,3,ABBA by ABBA,1975-04-21,11,7,"I Do, I Do, I Do, I Do, I Do","Love me or leave me, make your choice but beli..."
7,3,ABBA by ABBA,1975-04-21,11,8,Rock Me,"Rock me, give me that kick now\nRock me, show ..."
8,3,ABBA by ABBA,1975-04-21,11,9,Intermezzo No. 1,
9,3,ABBA by ABBA,1975-04-21,11,10,I’ve Been Waiting for You,"I, I've been in love before\nI thought I would..."


In [108]:
# In this case, we add two songs and drop the instrumental one ('Intermezzo No. 1')
df_abba['n_tracks'] = 12
df_abba['notes'] = np.nan
df_abba = df_abba.drop(index=8).reset_index(drop=True)

In [109]:
# 1st song to be added: 'Crazy World'
song_crazy_world = genius.search_song(title='Crazy World', artist='ABBA', get_full_info=True)

to_append = [3, df_abba.album[0], df_abba.release_date[0], df_abba.n_tracks_original[0], 11, song_crazy_world.title,
             song_crazy_world.lyrics, 13, '2001 CD edition bonus tracks']

to_append_series = pd.Series(to_append, index = df_abba.columns)
df_abba = df_abba.append(to_append_series, ignore_index=True)

Searching for "Crazy World" by ABBA...
Done.


In [110]:
# 2nd song: "Pick a Bale of Cotton"/"On Top of Old Smokey"/"Midnight Special" (medley)
song_medley = genius.search_song(title='Pick a Bale of Cotton"/"On Top of Old Smokey"/"Midnight Special',
                                 artist='ABBA', get_full_info=True)

to_append = [3, df_abba.album[0], df_abba.release_date[0], df_abba.n_tracks_original[0], 12, song_medley.title,
            song_medley.lyrics, 13, '2001 CD edition bonus tracks']

to_append_series = pd.Series(to_append, index = df_abba.columns)
df_abba = df_abba.append(to_append_series, ignore_index=True)

Searching for "Pick a Bale of Cotton"/"On Top of Old Smokey"/"Midnight Special" by ABBA...
Done.


In [111]:
df_abba.loc[:, 'id_track'] = range(1, 1+df_abba.shape[0]) # reset the id_track
df_abba

,id,album,release_date,n_tracks_original,id_track,track,lyrics,n_tracks,notes
0,3,ABBA by ABBA,1975-04-21,11,1,Mamma Mia,I've been cheated by you since I don't know wh...,12,NaN
1,3,ABBA by ABBA,1975-04-21,11,2,"Hey, Hey Helen","So at last, you're free\nIt's the way you want...",12,NaN
2,3,ABBA by ABBA,1975-04-21,11,3,Tropical Loveland,"Come to my loveland, wander along\nBeautiful g...",12,NaN
3,3,ABBA by ABBA,1975-04-21,11,4,S.O.S.,Where are those happy days? They seem so hard ...,12,NaN
4,3,ABBA by ABBA,1975-04-21,11,5,Man in the Middle,Did you see that man in the limousine?\nWith t...,12,NaN
5,3,ABBA by ABBA,1975-04-21,11,6,Bang-a-Boomerang,Making somebody happy is a question of give an...,12,NaN
6,3,ABBA by ABBA,1975-04-21,11,7,"I Do, I Do, I Do, I Do, I Do","Love me or leave me, make your choice but beli...",12,NaN
7,3,ABBA by ABBA,1975-04-21,11,8,Rock Me,"Rock me, give me that kick now\nRock me, show ...",12,NaN
8,3,ABBA by ABBA,1975-04-21,11,9,I’ve Been Waiting for You,"I, I've been in love before\nI thought I would...",12,NaN
9,3,ABBA by ABBA,1975-04-21,11,10,So Long,You think you're gonna make me softer with you...,12,NaN


## 4. Arrival
Released: 11 October 1976
- https://genius.com/albums/Abba/Arrival
- https://en.wikipedia.org/wiki/Arrival_(ABBA_album)

In [89]:
df_arrival = lyrics_df('Arrival', 4)
df_arrival

Searching for "Arrival" by ABBA...
https://genius.com/albums/Abba/Arrival


In [92]:
df_arrival

,id,album,release_date,n_tracks_original,id_track,track,lyrics
0,4,Arrival by ABBA,1976-10-11,10,1,When I Kissed the Teacher,Everybody screamed\nWhen I kissed the teacher\...
1,4,Arrival by ABBA,1976-10-11,10,2,Dancing Queen,"You can dance, you can jive\nHaving the time o..."
2,4,Arrival by ABBA,1976-10-11,10,3,"My Love, My Life",I've seen it on your face\nTells me more than ...
3,4,Arrival by ABBA,1976-10-11,10,4,Dum Dum Diddle,"I can hear how you work, practising hard\nPlay..."
4,4,Arrival by ABBA,1976-10-11,10,5,"Knowing Me, Knowing You",No more carefree laughter\nSilence ever after\...
5,4,Arrival by ABBA,1976-10-11,10,6,"Money, Money, Money","I work all night, I work all day, to pay the b..."
6,4,Arrival by ABBA,1976-10-11,10,7,That’s Me,Are you sure you wanna hear more?\nWhat if I a...
7,4,Arrival by ABBA,1976-10-11,10,8,Why Did It Have to Be Me?,"When you were lonely, you needed a man\nSomeon..."
8,4,Arrival by ABBA,1976-10-11,10,9,Tiger,"The city is a jungle, you'd better take care\n..."
9,4,Arrival by ABBA,1976-10-11,10,10,Arrival,3EmbedShare URLCopyEmbedCopy


In [93]:
# We add again 2 songs and drop 'Arrival', which is defined as a instrumental w/ vocalisations song
df_arrival['n_tracks'] = 11
df_arrival['notes'] = np.nan
df_arrival = df_arrival.drop(index=9).reset_index(drop=True)

In [94]:
# 1st song: 'Fernando'
song_fernando = genius.search_song(title='Fernando', artist='ABBA', get_full_info=True)

to_append = [4, df_arrival.album[0], df_arrival.release_date[0], df_arrival.n_tracks_original[0], 10,
             song_fernando.title, song_fernando.lyrics, df_arrival.n_tracks[0], '1997 CD edition bonus track']

to_append_series = pd.Series(to_append, index = df_arrival.columns)
df_arrival = df_arrival.append(to_append_series, ignore_index=True)

Searching for "Fernando" by ABBA...
Done.


In [95]:
# 2nd song: 'Happy Hawaii'
song_happy_hawaii = genius.search_song(title='Happy Hawaii', artist='ABBA', get_full_info=True)

to_append = [4, df_arrival.album[0], df_arrival.release_date[0], df_arrival.n_tracks_original[0], 11,
             song_happy_hawaii.title, song_happy_hawaii.lyrics, df_arrival.n_tracks[0], '2001 CD edition bonus tracks']

to_append_series = pd.Series(to_append, index = df_arrival.columns)
df_arrival = df_arrival.append(to_append_series, ignore_index=True)

Searching for "Happy Hawaii" by ABBA...
Done.


In [96]:
df_arrival

,id,album,release_date,n_tracks_original,id_track,track,lyrics,n_tracks,notes
0,4,Arrival by ABBA,1976-10-11,10,1,When I Kissed the Teacher,Everybody screamed\nWhen I kissed the teacher\...,11,NaN
1,4,Arrival by ABBA,1976-10-11,10,2,Dancing Queen,"You can dance, you can jive\nHaving the time o...",11,NaN
2,4,Arrival by ABBA,1976-10-11,10,3,"My Love, My Life",I've seen it on your face\nTells me more than ...,11,NaN
3,4,Arrival by ABBA,1976-10-11,10,4,Dum Dum Diddle,"I can hear how you work, practising hard\nPlay...",11,NaN
4,4,Arrival by ABBA,1976-10-11,10,5,"Knowing Me, Knowing You",No more carefree laughter\nSilence ever after\...,11,NaN
5,4,Arrival by ABBA,1976-10-11,10,6,"Money, Money, Money","I work all night, I work all day, to pay the b...",11,NaN
6,4,Arrival by ABBA,1976-10-11,10,7,That’s Me,Are you sure you wanna hear more?\nWhat if I a...,11,NaN
7,4,Arrival by ABBA,1976-10-11,10,8,Why Did It Have to Be Me?,"When you were lonely, you needed a man\nSomeon...",11,NaN
8,4,Arrival by ABBA,1976-10-11,10,9,Tiger,"The city is a jungle, you'd better take care\n...",11,NaN
9,4,Arrival by ABBA,1976-10-11,10,10,Fernando,"Can you hear the drums, Fernando?\nI remember,...",11,1997 CD edition bonus track


## 5. ABBA: The Album
Released: 11 October 1976
- https://genius.com/albums/Abba/Abba-the-album
- https://en.wikipedia.org/wiki/Arrival_(ABBA_album)

In [112]:
df_abba_album = lyrics_df('ABBA: The Album', 5)
df_abba

Searching for "ABBA: The Album" by ABBA...
https://genius.com/albums/Abba/Abba-the-album


,id,album,release_date,n_tracks_original,id_track,track,lyrics,n_tracks,notes
0,3,ABBA by ABBA,1975-04-21,11,1,Mamma Mia,I've been cheated by you since I don't know wh...,12,NaN
1,3,ABBA by ABBA,1975-04-21,11,2,"Hey, Hey Helen","So at last, you're free\nIt's the way you want...",12,NaN
2,3,ABBA by ABBA,1975-04-21,11,3,Tropical Loveland,"Come to my loveland, wander along\nBeautiful g...",12,NaN
3,3,ABBA by ABBA,1975-04-21,11,4,S.O.S.,Where are those happy days? They seem so hard ...,12,NaN
4,3,ABBA by ABBA,1975-04-21,11,5,Man in the Middle,Did you see that man in the limousine?\nWith t...,12,NaN
5,3,ABBA by ABBA,1975-04-21,11,6,Bang-a-Boomerang,Making somebody happy is a question of give an...,12,NaN
6,3,ABBA by ABBA,1975-04-21,11,7,"I Do, I Do, I Do, I Do, I Do","Love me or leave me, make your choice but beli...",12,NaN
7,3,ABBA by ABBA,1975-04-21,11,8,Rock Me,"Rock me, give me that kick now\nRock me, show ...",12,NaN
8,3,ABBA by ABBA,1975-04-21,11,9,I’ve Been Waiting for You,"I, I've been in love before\nI thought I would...",12,NaN
9,3,ABBA by ABBA,1975-04-21,11,10,So Long,You think you're gonna make me softer with you...,12,NaN


In [113]:
df_abba_album

,id,album,release_date,n_tracks_original,id_track,track,lyrics
0,5,ABBA: The Album by ABBA,1977-12-12,9,1,Eagle,They came flying from far away\nNow I'm under ...
1,5,ABBA: The Album by ABBA,1977-12-12,9,2,Take a Chance on Me,"If you change your mind, I'm the first in line..."
2,5,ABBA: The Album by ABBA,1977-12-12,9,3,"One Man, One Woman","No smiles, not a single word at the breakfast ..."
3,5,ABBA: The Album by ABBA,1977-12-12,9,4,The Name of the Game,"I've seen you twice, in a short time\nOnly a w..."
4,5,ABBA: The Album by ABBA,1977-12-12,9,5,Move On,"""They say a restless body can hide a peaceful ..."
5,5,ABBA: The Album by ABBA,1977-12-12,9,6,Hole in Your Soul,"You feel bad, let me tell you, we all get the ..."
6,5,ABBA: The Album by ABBA,1977-12-12,9,7,Thank You for the Music,"I'm nothing special, in fact I'm a bit of a bo..."
7,5,ABBA: The Album by ABBA,1977-12-12,9,8,I Wonder (Departure),"This park and these houses, old streets I have..."
8,5,ABBA: The Album by ABBA,1977-12-12,9,9,I’m a Marionette,"""You're so free,"" that's what everybody's tell..."


In [114]:
df_abba_album['n_tracks'] = 9
df_abba_album['notes'] = np.nan

## 6. Voulez-Vous
Released: 23 april 1979
- https://genius.com/albums/Abba/Voulez-vous
- https://en.wikipedia.org/wiki/Voulez-Vous

In [120]:
df_voulez_vous = lyrics_df('Voulez-Vous', 6)
df_voulez_vous

Searching for "Voulez-Vous" by ABBA...
https://genius.com/albums/Abba/Voulez-vous


,id,album,release_date,n_tracks_original,id_track,track,lyrics
0,6,Voulez-Vous by ABBA,1979-04-23,18,1,As Good As New,I'll never know why I had to go\nWhy I had to ...
1,6,Voulez-Vous by ABBA,1979-04-23,18,2,Voulez-Vous,People everywhere\nA sense of expectation hang...
2,6,Voulez-Vous by ABBA,1979-04-23,18,3,I Have a Dream,"I have a dream, a song to sing\nTo help me cop..."
3,6,Voulez-Vous by ABBA,1979-04-23,18,4,Angeleyes,Keep thinking about his angeleyes\nI keep thin...
4,6,Voulez-Vous by ABBA,1979-04-23,18,5,The King Has Lost His Crown,I believe your new girl turned you down\nAnd t...
5,6,Voulez-Vous by ABBA,1979-04-23,18,6,Does Your Mother Know,"You're so hot, teasing me\nSo you're blue, but..."
6,6,Voulez-Vous by ABBA,1979-04-23,18,7,If It Wasn’t for the Nights,"I got appointments, work I have to do\nKeeping..."
7,6,Voulez-Vous by ABBA,1979-04-23,18,8,Chiquitita,"Chiquitita, tell me what's wrong\nYou're encha..."
8,6,Voulez-Vous by ABBA,1979-04-23,18,9,Lovers (Live a Little Longer),Sit down and listen 'cause I've got good news ...
9,6,Voulez-Vous by ABBA,1979-04-23,18,10,Kisses of Fire,Lay your head on my chest so you hear every be...


In [121]:
# Adjustments
df_voulez_vous['n_tracks'] = 14
df_voulez_vous['notes'] = np.nan
df_voulez_vous.loc[df_voulez_vous.id_track==11, 'notes'] = '1997 CD edition bonus tracks'
df_voulez_vous.loc[df_voulez_vous.id_track==12, 'notes'] = 'The Definitive Collection 2001 CD edition bonus tracks'
df_voulez_vous.loc[df_voulez_vous.id_track==13, 'notes'] = 'The Definitive Collection 2001 CD edition bonus tracks'
df_voulez_vous.loc[df_voulez_vous.id_track==17, 'notes'] = '2010 deluxe edition (The Dynamic Album) bonus tracks'
df_voulez_vous = df_voulez_vous[~df_voulez_vous.id_track.isin([14, 15, 16, 18])]

In [123]:
# Reset index and id track column
df_voulez_vous.loc[:, 'id_track'] = range(1, 1+df_voulez_vous.shape[0])
df_voulez_vous = df_voulez_vous.reset_index(drop=True)
df_voulez_vous

,id,album,release_date,n_tracks_original,id_track,track,lyrics,n_tracks,notes
0,6,Voulez-Vous by ABBA,1979-04-23,18,1,As Good As New,I'll never know why I had to go\nWhy I had to ...,14,NaN
1,6,Voulez-Vous by ABBA,1979-04-23,18,2,Voulez-Vous,People everywhere\nA sense of expectation hang...,14,NaN
2,6,Voulez-Vous by ABBA,1979-04-23,18,3,I Have a Dream,"I have a dream, a song to sing\nTo help me cop...",14,NaN
3,6,Voulez-Vous by ABBA,1979-04-23,18,4,Angeleyes,Keep thinking about his angeleyes\nI keep thin...,14,NaN
4,6,Voulez-Vous by ABBA,1979-04-23,18,5,The King Has Lost His Crown,I believe your new girl turned you down\nAnd t...,14,NaN
5,6,Voulez-Vous by ABBA,1979-04-23,18,6,Does Your Mother Know,"You're so hot, teasing me\nSo you're blue, but...",14,NaN
6,6,Voulez-Vous by ABBA,1979-04-23,18,7,If It Wasn’t for the Nights,"I got appointments, work I have to do\nKeeping...",14,NaN
7,6,Voulez-Vous by ABBA,1979-04-23,18,8,Chiquitita,"Chiquitita, tell me what's wrong\nYou're encha...",14,NaN
8,6,Voulez-Vous by ABBA,1979-04-23,18,9,Lovers (Live a Little Longer),Sit down and listen 'cause I've got good news ...,14,NaN
9,6,Voulez-Vous by ABBA,1979-04-23,18,10,Kisses of Fire,Lay your head on my chest so you hear every be...,14,NaN


## 7. Super Trouper
Released: 3 November 1980
- https://genius.com/albums/Abba/Super-trouper
- https://en.wikipedia.org/wiki/Super_Trouper_(album)

In [124]:
df_super_trouper = lyrics_df('Super Trouper', 7)
df_super_trouper

Searching for "Super Trouper" by ABBA...
https://genius.com/albums/Abba/Super-trouper


,id,album,release_date,n_tracks_original,id_track,track,lyrics
0,7,Super Trouper by ABBA,1980-11-03,14,1,Super Trouper,Super Trouper beams are gonna blind me\nBut I ...
1,7,Super Trouper by ABBA,1980-11-03,14,2,The Winner Takes It All,I don't wanna talk\nAbout things we've gone th...
2,7,Super Trouper by ABBA,1980-11-03,14,3,On and On and On,"I was at a party and this fella said to me\n""S..."
3,7,Super Trouper by ABBA,1980-11-03,14,4,"Andante, Andante","Take it easy with me, please\nTouch me gently ..."
4,7,Super Trouper by ABBA,1980-11-03,14,5,Me and I,Sometimes when I'm mad\nThere's a part of me t...
5,7,Super Trouper by ABBA,1980-11-03,14,6,Happy New Year,No more champagne\nAnd the fireworks are throu...
6,7,Super Trouper by ABBA,1980-11-03,14,7,Our Last Summer,The summer air was soft and warm\nThe feeling ...
7,7,Super Trouper by ABBA,1980-11-03,14,8,The Piper,They came from the hills\nAnd they came from t...
8,7,Super Trouper by ABBA,1980-11-03,14,9,Lay All Your Love on Me,I wasn't jealous before we met\nNow every woma...
9,7,Super Trouper by ABBA,1980-11-03,14,10,The Way Old Friends Do (Live),You and I can share the silence\nFinding comfo...


In [125]:
# Adjustments
df_super_trouper['n_tracks'] = 12
df_super_trouper['notes'] = np.nan
df_super_trouper = df_super_trouper.copy().iloc[:-2]
df_super_trouper.loc[df_super_trouper.id_track==11, 'notes'] = '1997 CD edition bonus tracks'
df_super_trouper.loc[df_super_trouper.id_track==12, 'notes'] = '1997 CD edition bonus tracks'

In [126]:
df_super_trouper

,id,album,release_date,n_tracks_original,id_track,track,lyrics,n_tracks,notes
0,7,Super Trouper by ABBA,1980-11-03,14,1,Super Trouper,Super Trouper beams are gonna blind me\nBut I ...,12,NaN
1,7,Super Trouper by ABBA,1980-11-03,14,2,The Winner Takes It All,I don't wanna talk\nAbout things we've gone th...,12,NaN
2,7,Super Trouper by ABBA,1980-11-03,14,3,On and On and On,"I was at a party and this fella said to me\n""S...",12,NaN
3,7,Super Trouper by ABBA,1980-11-03,14,4,"Andante, Andante","Take it easy with me, please\nTouch me gently ...",12,NaN
4,7,Super Trouper by ABBA,1980-11-03,14,5,Me and I,Sometimes when I'm mad\nThere's a part of me t...,12,NaN
5,7,Super Trouper by ABBA,1980-11-03,14,6,Happy New Year,No more champagne\nAnd the fireworks are throu...,12,NaN
6,7,Super Trouper by ABBA,1980-11-03,14,7,Our Last Summer,The summer air was soft and warm\nThe feeling ...,12,NaN
7,7,Super Trouper by ABBA,1980-11-03,14,8,The Piper,They came from the hills\nAnd they came from t...,12,NaN
8,7,Super Trouper by ABBA,1980-11-03,14,9,Lay All Your Love on Me,I wasn't jealous before we met\nNow every woma...,12,NaN
9,7,Super Trouper by ABBA,1980-11-03,14,10,The Way Old Friends Do (Live),You and I can share the silence\nFinding comfo...,12,NaN


## 8. The Visitors
Released: 30 November 1981
- https://genius.com/albums/Abba/The-visitors
- https://en.wikipedia.org/wiki/The_Visitors_(ABBA_album)

In [127]:
df_visitors = lyrics_df('The Visitors', 8)
df_visitors

Searching for "The Visitors" by ABBA...
https://genius.com/albums/Abba/The-visitors


,id,album,release_date,n_tracks_original,id_track,track,lyrics
0,8,The Visitors by ABBA,1981-11-30,18,1,The Visitors,I hear the doorbell ring and suddenly the pani...
1,8,The Visitors by ABBA,1981-11-30,18,2,Head Over Heels,I have a very good friend\nThe kind of girl wh...
2,8,The Visitors by ABBA,1981-11-30,18,3,When All Is Said and Done,"Here's to us, one more toast, and then we'll p..."
3,8,The Visitors by ABBA,1981-11-30,18,4,Soldiers,Do I hear what I think I'm hearing?\nDo I see ...
4,8,The Visitors by ABBA,1981-11-30,18,5,I Let the Music Speak,"I'm hearing images, I'm seeing songs\nNo poet ..."
5,8,The Visitors by ABBA,1981-11-30,18,6,One of Us,"They passed me by, all of those great romances..."
6,8,The Visitors by ABBA,1981-11-30,18,7,Two for the Price of One,He had what you might call a trivial occupatio...
7,8,The Visitors by ABBA,1981-11-30,18,8,Slipping Through My Fingers,"Schoolbag in hand, she leaves home in the earl..."
8,8,The Visitors by ABBA,1981-11-30,18,9,Like an Angel Passing Through My Room,Long awaited darkness falls\nCasting shadows o...
9,8,The Visitors by ABBA,1981-11-30,18,10,Should I Laugh or Cry,"""Noll... ett, två, tre""\n\nHe stands towering ..."


In [128]:
# Adjustments
df_visitors['n_tracks'] = 16
df_visitors['notes'] = np.nan

df_visitors = df_visitors[~df_visitors.id_track.isin([11, 12])] # Drop Spanish songs

df_visitors.loc[df_visitors.id_track==10, 'notes'] = '1997 CD edition bonus tracks'
df_visitors.loc[df_visitors.id_track==14, 'notes'] = '1997 CD edition bonus tracks'
df_visitors.loc[df_visitors.id_track==16, 'notes'] = '1997 CD edition bonus tracks'
df_visitors.loc[df_visitors.id_track==17, 'notes'] = '1997 CD edition bonus tracks'

df_visitors.loc[df_visitors.id_track==15, 'notes'] = '2001 CD edition bonus tracks'
df_visitors.loc[df_visitors.id_track==13, 'notes'] = '2012 deluxe edition (The Final Album) bonus tracks'
df_visitors.loc[df_visitors.id_track==18, 'notes'] = '2012 deluxe edition (The Final Album) bonus tracks'

/Users/dossim/anaconda3/envs/myenv_python38/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [130]:
df_visitors = df_visitors.reset_index(drop=True)
df_visitors

,id,album,release_date,n_tracks_original,id_track,track,lyrics,n_tracks,notes
0,8,The Visitors by ABBA,1981-11-30,18,1,The Visitors,I hear the doorbell ring and suddenly the pani...,16,NaN
1,8,The Visitors by ABBA,1981-11-30,18,2,Head Over Heels,I have a very good friend\nThe kind of girl wh...,16,NaN
2,8,The Visitors by ABBA,1981-11-30,18,3,When All Is Said and Done,"Here's to us, one more toast, and then we'll p...",16,NaN
3,8,The Visitors by ABBA,1981-11-30,18,4,Soldiers,Do I hear what I think I'm hearing?\nDo I see ...,16,NaN
4,8,The Visitors by ABBA,1981-11-30,18,5,I Let the Music Speak,"I'm hearing images, I'm seeing songs\nNo poet ...",16,NaN
5,8,The Visitors by ABBA,1981-11-30,18,6,One of Us,"They passed me by, all of those great romances...",16,NaN
6,8,The Visitors by ABBA,1981-11-30,18,7,Two for the Price of One,He had what you might call a trivial occupatio...,16,NaN
7,8,The Visitors by ABBA,1981-11-30,18,8,Slipping Through My Fingers,"Schoolbag in hand, she leaves home in the earl...",16,NaN
8,8,The Visitors by ABBA,1981-11-30,18,9,Like an Angel Passing Through My Room,Long awaited darkness falls\nCasting shadows o...,16,NaN
9,8,The Visitors by ABBA,1981-11-30,18,10,Should I Laugh or Cry,"""Noll... ett, två, tre""\n\nHe stands towering ...",16,1997 CD edition bonus tracks


## Concatenate all the 8 datasets and save the resulting df

In [133]:
df = pd.concat([
    df_ring_ring,
    df_waterloo,
    df_abba,
    df_arrival,
    df_abba_album,
    df_voulez_vous,
    df_super_trouper,
    df_visitors
])

df.reset_index(drop=True, inplace=True)

In [134]:
df.shape

(99, 9)

In [137]:
df.to_csv('../data/df_abba_lyrics.csv', index=False)